In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn import metrics
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn import metrics
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from sklearn.ensemble import RandomForestClassifier
import gc

## Data Prep

In [3]:
# train_X = pd.read_csv('music_data/train_X.csv')
train_X = pd.read_csv('music_data/train_X_date_converted.csv')
train_y = pd.read_csv('music_data/train_Y.csv')

# val_X = pd.read_csv('music_data/valid_X.csv')
val_X = pd.read_csv('music_data/valid_X_date_converted.csv')
val_y = pd.read_csv('music_data/valid_Y.csv')

# test_X = pd.read_csv('music_data/test_X.csv')
test_X = pd.read_csv('music_data/test_X_date_converted.csv')
test_y = pd.read_csv('music_data/test_Y.csv')

# train_tiny_X = pd.read_csv('music_data/train_tiny_X.csv')
train_tiny_X = pd.read_csv('music_data/train_tiny_X_date_converted.csv')
train_tiny_y = pd.read_csv('music_data/train_tiny_Y.csv')

# val_tiny_X = val_X[:1000].copy()
val_tiny_X = pd.read_csv('music_data/val_tiny_X_date_converted.csv')
val_tiny_y = val_y[:1000].copy()

## Random Forest

In [4]:
catCols = ['msno', 'song_id', 'source_screen_name', 'source_system_tab', 'source_type', 'genre_ids', 'artist_name', 
           'composer', 'lyricist', 'language', 'city', 'gender', 'registered_via']
numCols = ['bd', 'song_length']

In [12]:
data_scaler = StandardScaler()
train_num = data_scaler.fit_transform(train_tiny_X[numCols])
val_num = data_scaler.transform(val_tiny_X[numCols])
train_num_df = pd.DataFrame(train_num,columns = numCols)
train_num_df = train_num_df.reset_index(drop=True)
val_num_df = pd.DataFrame(val_num,columns = numCols)
val_num_df = val_num_df.reset_index(drop=True)

train_cat = train_tiny_X[catCols]
one_hot = OneHotEncoder(handle_unknown='ignore')
one_hot.fit(train_cat)
feature_names = [str(i) for i in list(one_hot.get_feature_names())]
for i in range(4):
    if i == 0:
        train_piece = one_hot.transform(train_cat[i*1000:(i+1)*1000]).toarray()
        val_piece = one_hot.transform(val_tiny_X[catCols]).toarray()
    elif i == 3:
        train_piece = np.vstack((train_piece,one_hot.transform(train_cat[3000:]).toarray()))      
    else:
        train_piece = np.vstack((train_piece,one_hot.transform(train_cat[i*1000:(i+1)*1000]).toarray()))

train_cat_df = pd.DataFrame(train_piece,columns = catCols)
val_cat_df = pd.DataFrame(val_piece,columns = catCols)
test_cat_df = pd.DataFrame(test_piece,columns = catCols)
newcatCols = list(train_cat_df.columns)

try_train_X = pd.concat([train_num_df,train_cat_df],axis = 1)
try_val_X = pd.concat([val_num_df,val_cat_df],axis = 1)
try_test_X = pd.concat([test_num_df,test_cat_df],axis = 1)

ValueError: Shape of passed values is (10000, 21095), indices imply (10000, 13)

In [10]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 10, stop = 100, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 10)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
rf_random = RandomizedSearchCV(estimator = RandomForestClassifier(random_state=0), param
                               _distributions = random_grid, n_iter = 20, cv = 3, verbose=1, random_state=0)
rf_random.fit(train_tiny_X, train_tiny_y)

Fitting 3 folds for each of 20 candidates, totalling 60 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


ValueError: multiclass-multioutput is not supported

In [ ]:
print(rf_random.best_params_)

In [ ]:
clf = RandomForestClassifier(n_estimators=500,random_state=0,min_samples_split=10)
clf.fit(train_tiny_X, train_tiny_y)
print(classification_report(val_tiny_y, clf.predict(val_tiny_X)>0.5))
print(roc_auc_score(val_tiny_y, clf.predict(val_tiny_X)))

In [9]:
gc.collect()

567